# Daizika Edgar API Demo
## Prerequisite: httplib2 https://github.com/httplib2/httplib2

In [ ]:
import httplib2
from urllib.parse import urlencode, quote
import json

In [ ]:
# Set the base_url
base_url = 'https://api.daizika.com/edgar/test/'

# Set your api key
x_api_key = 'XXXXXXXXXXXX'

# Set your headers
headers = {'Accept': 'application/json'
         , 'Content-Type': 'application/json; charset=UTF-8'
         , 'x-api-key': x_api_key} 

In [ ]:
def get_all_fillings(base_url, headers, date, offset=0, size=25, form_type=None):    
    params = {u"date": date, u"offset": offset, u"size":size}
    if form_type is not None:
        params[u"form_type"] = form_type        
    query_params = urlencode(params, quote_via=quote)
    request_url = "{}filling?{}".format(base_url, query_params)
    print('Requesting: {}'.format(request_url))
    
    # Create the request processor
    http = httplib2.Http()
    response, content = http.request(request_url, 'GET', headers=headers)

    # Retrieve the JSON 
    data = json.loads(content)

    found = False
    # get the results
    if data is not None:
        found = True
    
    return found, data

def get_cik_fillings(base_url, headers, cik, start=0, size=25, form_type=None):    
    params = {u"start": start, u"size":size}
    if form_type is not None:
        params = {u"form_type": form_type}        
    query_params = urlencode(params, quote_via=quote)
    request_url = "{}{}/fillings?{}".format(base_url, cik, query_params)

    # Create the request processor
    http = httplib2.Http()
    response, content = http.request(request_url, 'GET', headers=headers)

    # Retrieve the JSON 
    data = json.loads(content)
    
    found = True
    # get the results
    if 'message' in data:
        found = False
    
    return found, data

def get_cik_fillings_data(base_url, headers, cik, form, year, qtr):    
    params = {u"form": form, u"year":year, u"qtr": qtr}
    query_params = urlencode(params, quote_via=quote)
    request_url = "{}{}/fillings/data?{}".format(base_url, cik, query_params)

    # Create the request processor
    http = httplib2.Http()
    response, content = http.request(request_url, 'GET', headers=headers)

    # Retrieve the JSON 
    data = json.loads(content)
    
    found = True
    # get the results
    if 'message' in data:
        found = False
    
    return found, data

def get_cik_fillings_statement(base_url, headers, cik, form, year, qtr, doc_id):    
    params = {u"form": form, u"year":year, u"qtr": qtr, u"doc_id": doc_id}
    query_params = urlencode(params, quote_via=quote)
    request_url = "{}{}/fillings/data/statement?{}".format(base_url, cik, query_params)

    # Create the request processor
    http = httplib2.Http()
    response, content = http.request(request_url, 'GET', headers=headers)

    # Retrieve the JSON 
    data = json.loads(content)
    
    found = True
    # get the results
    if 'message' in data:
        found = False
    
    return found, data


In [ ]:
# Querying for the CIK
date = "2020-06-01"
offset = 0
size = 25
form_type = '10-K'
found, data = get_all_fillings(base_url, headers, date=date, offset=offset, size=size, form_type=None)

if found:    
    print(data)
else:
    print("No results found")

In [ ]:
# Querying for the CIK
cik_to_query = 1459417
form_type = '10-K'
found, data = get_cik_fillings(base_url, headers, cik=cik_to_query, start=10, size=20, form_type=form_type)

if found:    
    for each_filling in data:            
        print(each_filling)
else:
    print("No results found")

In [ ]:
# Getting the details on 10-K fillings
cik_to_query = 1459417 #1594109
form_type = '10-K' 
year = 2014
qtr = 0
found, data = get_cik_fillings_data(base_url, headers, cik=cik_to_query, form=form_type, year=year, qtr=qtr)

if found:    
    for each_filling in data:            
        print('{}|{}|{}'.format(each_filling['doc_id'], each_filling['doc_name'], each_filling['doc_label']))
else:
    print("No results found")

In [ ]:
def print_items(level, items):    
    print('{}{}'.format(level, items['name']))
    if items['abstract'] == 'false' and len(items['data']) > 0:
        for each_val in items['data']:
            if len(each_val['dimensions']) == 0:
                print('...{}{}|{}|{}'.format(level, each_val['endInstant'], each_val['value'], each_val['unitRef']))
    if 'items' in items and len(items['items']) > 0:
        level += '...'
        for key in items['items']:
            print_items(level, items['items'][key])

# Getting the BalanceSheet on 10-K fillings
cik_to_query = 1459417
form_type = '10-K' 
year = 2014
qtr = 0
doc_id = 3 #doc_id is retrieved from the previous API call

found, data = get_cik_fillings_statement(base_url, headers, cik=cik_to_query, form=form_type, year=year, qtr=qtr, doc_id=doc_id)

if found:    
    print('{}'.format(data['label']))
    level = '...'
    for key in data['items']:
        print_items(level, data['items'][key])
else:
    print("No results found")